# Usando outras linguagens

<BR></BR>
## 1. [C](#C)
<BR></BR>
## 2. [Python](#Python)
<BR></BR>
## 3. [R](#R)
<BR></BR>
## 3. [MATLAB](#MATLAB)

Julia é uma linguagem de programação nova e como toda linguagem nova não tem tantos pacotes quanto linguagens mais antigas. Pensando nisso Julia criou formas fáceis de chamar códigos de outras linguagens mais maduras. Isso facilita muito a migração para Julia de outras linguagens. Você pode continuar usando aquela biblioteca que você usava na sua antiga linguagem.
Nessa parte vou dar uma breve introdução de como chamar funções do C e focar mais em Python e R.

# C

Existem muitas bibliotecas maduras de alta qualidade para computação numérica já escritas em C e Fortran. Para permitir o uso fácil deste código existente, Julia torna simples e eficiente chamar funções C e Fortran.

Também existe a possibilidade de chamar Julia do C.

In [ ]:
# A função ccall
t = ccall( (:time, "libc"), Int32, ()) # Tempo em segundos

In [ ]:
str = "Hello, World! \n"
ccall((:printf, "libc"), Int,
        (Ptr{Cchar}, Cint),
        str, sizeof(str));

# Python

A integração com Python é feita através da biblioteca PyCall, que proporciona uma maneira simples de chamar funções e trocar estruturas entre as duas linguagens. 

Também é possível chamar código do Julia do Python.

In [3]:
# Usando uma biblioteca do Python
using PyCall
@pyimport numpy as np
np.ones((2,10));

In [4]:
@pyimport numpy.random as nr
A = nr.random((3,10));

In [12]:
@pyimport scipy.stats as st
s = st.norm(0, 1)

PyCall.PyObject

In [19]:
# Outra forma de chamar Python
x = [1, 2, 3]
py"sum($(x))"

6

In [20]:
# Também é possível usar múltiplas linhas mas nesse caso o retorno é nothing
py"""
import numpy as np
y = np.array( [11,12,13] )
print(y-$x)
"""

[10 10 10]


# R

Mesmo Julia tendo já vários pacotes, R tem cerca de 10 mil pacotes. Então a integração com R é essencial, principalmente para quem é de estatística.

In [6]:
using CategoricalArrays
using RCall

x = randn(10)
R"t.test($x)"

INFO: Precompiling module RCall.
INFO: Using R installation at /usr/lib/R
ERROR: LoadError: LoadError: UndefVarError: NullableCategoricalArray not defined
Stacktrace:
 [1] include_from_node1(::String) at ./loading.jl:576
 [2] include(::String) at ./sysimg.jl:14
 [3] include_from_node1(::String) at ./loading.jl:576
 [4] include(::String) at ./sysimg.jl:14
 [5] anonymous at ./<missing>:2
while loading /home/jrun/.julia/v0.6/RCall/src/convert/categorical.jl, in expression starting on line 11
while loading /home/jrun/.julia/v0.6/RCall/src/RCall.jl, in expression starting on line 47


LoadError: [91mFailed to precompile RCall to /mnt/juliabox/.julia/lib/v0.6/RCall.ji.[39m

In [17]:
x = [3, 9, 4]

R"print(mean($x))";

[1] 5.333333


In [19]:
# Assim como em Python podemos usar """ nesse caso ele retorna objeto de R
R"""
a <- c(3, 5, 6)
A <- a * 2
print(A)
print(sample(1:10))
""";

[1]  6 10 12
 [1]  1  7  8  2  9  4 10  6  3  5


## rcall
Assim como C tem a função ccall() e Python o pycall() o R tem a rcall()

In [3]:
s = rcall(:sum, Float64[3.0, 4.0, 6.0])

LoadError: [91mUndefVarError: rcall not defined[39m

In [4]:
@show typeof(s)
# Posso manipular s?
sj = rcopy(s)
@show typeof(sj)

LoadError: [91mUndefVarError: s not defined[39m

### @rput e @rget

Esses macros transferem variáveis entre ambientes R e Julia.

In [32]:
A = rand(2,2)
@show A
@rput A
A[1,1] =10
@show A
R"A"

A = [0.0485442 0.643239; 0.243206 0.000374923]
A = [10.0 0.643239; 0.243206 0.000374923]


RCall.RObject{RCall.RealSxp}
           [,1]         [,2]
[1,] 0.04854425 0.6432386500
[2,] 0.24320649 0.0003749232


In [31]:
R"z = det(A)"
R"print(z)"
@rget z
@show z
@show typeof(z);

[1] 0.005446402
z = 0.005446401650316683
typeof(z) = Float64


# MATLAB

O pacote MATLAB.jl é uma interface para utilizar o MATLAB de Julia. Infelizmente não dá para usar aqui no JuliaBox mas vou colocar alguns exemplos.

Existem 3 formas de acessar o MATLAB de Julia, bem similares as anteriores:

- `mat""` assim como os anteriores `py""` e `R""` permite escrever código em MATLAB e usar variáveis de julia usando interpolação
- `@matlab` macro, junto com `@mput` and `@mget`, traduz de MATLAB para Julia
- `mxcall` usado para chamar uma função em MATLAB

## Usando mat""

```julia
using MATLAB

x = linspace(0, 4π, 100)
mat"plot($x, cos($x))" # usando uma função do MATLAB
```

```julia
y = [1,2,3]
mat"""
    z = ones(3,1)
    $u = $y + $z
"""
println(u) # A variável u fica acessíveis do Julia
```

## Usando @matlab

```julia
x = linspace(0, 4π, 100)
@mput x                 # Coloca o vetor x no ambiente do MATLAB
@matlab plot(x, cos(x)) # Chamando uma função do MATLAB
```

```julia
y = linspace(-5.0, 5.0, 100)
@mput y
@matlab begin
    (xx, yy) = meshgrid(x, y)  # ==> MATLAB: [xx, yy] = meshgrid(x, y)
    mesh(xx, yy, xx.^2 + yy.^2)
end
```